In [1]:
import org.apache.spark.sql.functions.{col,lit,expr}
val spark_home = "/Users/fanzhenxin/bigData/spark-2.4.4-bin-hadoop2.7"
val data_path = spark_home+"/data"
val df_path = data_path+"/retail-data/by-day/2010-12-01.csv"
val df = spark.read.format("csv")
    .option("header","true")
    .option("inferSchema","true")
    .load(df_path)
df.createOrReplaceTempView("dfTable")

spark_home = /Users/fanzhenxin/bigData/spark-2.4.4-bin-hadoop2.7
data_path = /Users/fanzhenxin/bigData/spark-2.4.4-bin-hadoop2.7/data
df_path = /Users/fanzhenxin/bigData/spark-2.4.4-bin-hadoop2.7/data/retail-data/by-day/2010-12-01.csv
df = [InvoiceNo: string, StockCode: string ... 6 more fields]


[InvoiceNo: string, StockCode: string ... 6 more fields]

In [2]:
import org.apache.spark.sql.functions.coalesce

//df.select(coalesce(col("Description"),
//                  col("CustomerId"))).show()



In [3]:
%%SQL
SELECT
ifnull(null,'return_value') a,
nullif('value','value') b,
nvl(null,'return_value') c,
nvl2(null,'return_value','else_value') d
from dfTable LIMIT 1

+------------+----+------------+----------+
|           a|   b|           c|         d|
+------------+----+------------+----------+
|return_value|null|return_value|else_value|
+------------+----+------------+----------+



In [4]:
df.na.fill("All Null values become this string").show()

+---------+---------+--------------------+--------+-------------------+---------+----------+--------------+
|InvoiceNo|StockCode|         Description|Quantity|        InvoiceDate|UnitPrice|CustomerID|       Country|
+---------+---------+--------------------+--------+-------------------+---------+----------+--------------+
|   536365|   85123A|WHITE HANGING HEA...|       6|2010-12-01 08:26:00|     2.55|   17850.0|United Kingdom|
|   536365|    71053| WHITE METAL LANTERN|       6|2010-12-01 08:26:00|     3.39|   17850.0|United Kingdom|
|   536365|   84406B|CREAM CUPID HEART...|       8|2010-12-01 08:26:00|     2.75|   17850.0|United Kingdom|
|   536365|   84029G|KNITTED UNION FLA...|       6|2010-12-01 08:26:00|     3.39|   17850.0|United Kingdom|
|   536365|   84029E|RED WOOLLY HOTTIE...|       6|2010-12-01 08:26:00|     3.39|   17850.0|United Kingdom|
|   536365|    22752|SET 7 BABUSHKA NE...|       2|2010-12-01 08:26:00|     7.65|   17850.0|United Kingdom|
|   536365|    21730|GLASS S

In [5]:
println(df.count())
println(df.na.drop("any",Seq("CustomerID")).count())

3108
1968


In [6]:
val nullDF = df.filter('CustomerID.isNull)

// nullDF.na.fill("any","NA Value")
// .select("InvoiceNo","StockCode","Description","CustomerID")
// .show(5)

nullDF = [InvoiceNo: string, StockCode: string ... 6 more fields]


[InvoiceNo: string, StockCode: string ... 6 more fields]

In [7]:
df.na.fill("TEST").where(col("CustomerID").isNull).select("CustomerID").show()

+----------+
|CustomerID|
+----------+
|      null|
|      null|
|      null|
|      null|
|      null|
|      null|
|      null|
|      null|
|      null|
|      null|
|      null|
|      null|
|      null|
|      null|
|      null|
|      null|
|      null|
|      null|
|      null|
|      null|
+----------+
only showing top 20 rows



In [8]:
val df2 = df.na.fill(100:Double,Seq("CustomerID"))
df2.where(col("CustomerID")===100)
.select("CustomerID").show()

+----------+
|CustomerID|
+----------+
|     100.0|
|     100.0|
|     100.0|
|     100.0|
|     100.0|
|     100.0|
|     100.0|
|     100.0|
|     100.0|
|     100.0|
|     100.0|
|     100.0|
|     100.0|
|     100.0|
|     100.0|
|     100.0|
|     100.0|
|     100.0|
|     100.0|
|     100.0|
+----------+
only showing top 20 rows



df2 = [InvoiceNo: string, StockCode: string ... 6 more fields]


[InvoiceNo: string, StockCode: string ... 6 more fields]

In [9]:
df.printSchema

root
 |-- InvoiceNo: string (nullable = true)
 |-- StockCode: string (nullable = true)
 |-- Description: string (nullable = true)
 |-- Quantity: integer (nullable = true)
 |-- InvoiceDate: timestamp (nullable = true)
 |-- UnitPrice: double (nullable = true)
 |-- CustomerID: double (nullable = true)
 |-- Country: string (nullable = true)



In [10]:
val fillColValues = Map("CustomerID"->"NullVal")
df.na.fill(fillColValues).select("CustomerID").where(col("CustomerID")==="NullVal")
.show()

+----------+
|CustomerID|
+----------+
+----------+



fillColValues = Map(CustomerID -> NullVal)


Map(CustomerID -> NullVal)

In [11]:
df.na.replace("Description",Map(""->"UNKNOWN"))
.where(col("Description")==="UNKNOWN")
.select("Description")
.show()

+-----------+
|Description|
+-----------+
+-----------+



In [12]:
import org.apache.spark.sql.functions.struct
val complexDF = df.select(struct("Description","InvoiceNo").alias("complex"))
complexDF.createOrReplaceTempView("complexDF")

complexDF = [complex: struct<Description: string, InvoiceNo: string>]


[complex: struct<Description: string, InvoiceNo: string>]

In [13]:
import org.apache.spark.sql.functions.split

df.select(split(col("Description")," ")).show(2)

+---------------------+
|split(Description,  )|
+---------------------+
| [WHITE, HANGING, ...|
| [WHITE, METAL, LA...|
+---------------------+
only showing top 2 rows



In [14]:
df.select(split(col("Description")," ").alias("array_col"))
.selectExpr("array_col[0]").show(2)

+------------+
|array_col[0]|
+------------+
|       WHITE|
|       WHITE|
+------------+
only showing top 2 rows



In [15]:
import org.apache.spark.sql.functions.size

df.select(size(split(col("Description")," "))).show(2)

+---------------------------+
|size(split(Description,  ))|
+---------------------------+
|                          5|
|                          3|
+---------------------------+
only showing top 2 rows



In [16]:
import org.apache.spark.sql.functions.array_contains
df.select(array_contains(split(col("Description")," "),"WHITE"))
.show(2)

+--------------------------------------------+
|array_contains(split(Description,  ), WHITE)|
+--------------------------------------------+
|                                        true|
|                                        true|
+--------------------------------------------+
only showing top 2 rows



In [17]:
import org.apache.spark.sql.functions.{split,explode}

df.withColumn("splitted",split(col("Description")," "))
  .withColumn("exploded",explode(col("splitted")))
  .select("Description","InvoiceNo","exploded","splitted").show(2,false)

+----------------------------------+---------+--------+----------------------------------------+
|Description                       |InvoiceNo|exploded|splitted                                |
+----------------------------------+---------+--------+----------------------------------------+
|WHITE HANGING HEART T-LIGHT HOLDER|536365   |WHITE   |[WHITE, HANGING, HEART, T-LIGHT, HOLDER]|
|WHITE HANGING HEART T-LIGHT HOLDER|536365   |HANGING |[WHITE, HANGING, HEART, T-LIGHT, HOLDER]|
+----------------------------------+---------+--------+----------------------------------------+
only showing top 2 rows



In [18]:
df.withColumn("splitted",split(col("Description")," "))
.limit(2).select("splitted").show(2,false)

+----------------------------------------+
|splitted                                |
+----------------------------------------+
|[WHITE, HANGING, HEART, T-LIGHT, HOLDER]|
|[WHITE, METAL, LANTERN]                 |
+----------------------------------------+



In [19]:
df.withColumn("splitted",split(col("Description")," "))
.limit(1).select("splitted")
.withColumn("exploded",explode(col("splitted")))
.select("splitted","exploded").show(false)

+----------------------------------------+--------+
|splitted                                |exploded|
+----------------------------------------+--------+
|[WHITE, HANGING, HEART, T-LIGHT, HOLDER]|WHITE   |
|[WHITE, HANGING, HEART, T-LIGHT, HOLDER]|HANGING |
|[WHITE, HANGING, HEART, T-LIGHT, HOLDER]|HEART   |
|[WHITE, HANGING, HEART, T-LIGHT, HOLDER]|T-LIGHT |
|[WHITE, HANGING, HEART, T-LIGHT, HOLDER]|HOLDER  |
+----------------------------------------+--------+



In [20]:
import org.apache.spark.sql.functions.map

df.select(map(col("Description"),
             col("InvoiceNo")).alias("complex_map"))
.selectExpr("explode(complex_map)").show(2)

+--------------------+------+
|                 key| value|
+--------------------+------+
|WHITE HANGING HEA...|536365|
| WHITE METAL LANTERN|536365|
+--------------------+------+
only showing top 2 rows

